# 2D Function with discontinuity line

In [ ]:
using Revise
using Quantics
import Quantics: QuanticsInd
using Plots
import TensorCrossInterpolation as TCI

newaxis = [CartesianIndex()]

In [ ]:
using ITensors

In [ ]:
# TCI with R bits
function create_func(θ, R)
    g = Quantics.DiscretizedGrid{2}(R)

    function q_to_n(q::Vector{QuanticsInd{2}})::NTuple{2, Float64}
        return Quantics.originalcoordinate(g, q)
    end

    function func_q(q::Vector{QuanticsInd{2}})::Float64
        p = q_to_n(q)
        return p[2]/p[1] < tan(θ) ? 1.0 : 0.0
    end

    return q_to_n, func_q, g
end

R = 10
q2n, fq, g = create_func(π/6, R)
;

In [ ]:
fI = x->fq(QuanticsInd{2}.(x))
localdims = fill(4, R)

firstpivot = ones(Int, R)
@show fI(firstpivot)
firstpivot = TCI.optfirstpivot(fI, localdims, firstpivot)
@show fI(firstpivot)

In [ ]:
qtt, ranks, errors = TCI.crossinterpolate2(
    Float64, fI, localdims,
    [firstpivot], tolerance=1e-8, maxiter=10, verbosity=1
)

In [ ]:
coordinates = [(x, 0.5) for x in collect(LinRange(0, 1, 2^R + 1)[1:(end - 1)])]
gridpoints = [Quantics.gridpoint(g, c) for c in coordinates]
qs = [Quantics.to_quantics(g, p) for p in gridpoints]

reconsts = [TCI.evaluate(qtt, convert.(Int, q)) for q in qs]
refs = fq.(qs)
;

In [ ]:
p = plot(yaxis=:log, ylims=(1e-10,1e+4))
plot!(p, abs.(reconsts), marker=:x)
plot!(p, abs.(refs), marker=:+)
plot!(p, abs.(refs .- reconsts))

In [ ]:
f = collect(LinRange(0, 0.1, 100)[1:end-1])

cs_2d = collect(Iterators.product(f, f))
gridpoints_2d = [Quantics.gridpoint(g, c) for c in cs_2d]
qs_2d = [Quantics.to_quantics(g, p) for p in gridpoints_2d]

vals = [TCI.evaluate(qtt, convert.(Int, q)) for q in qs_2d]
;

In [ ]:
heatmap(log.(abs.(vals)))